# Exercise 1.5.2 - Mean Average Precision (mAP)
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Implement the Mean Average Precision (mAP) metric;
* To do so, create a function to calculate the [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) scores;
* Visualise the Precision-Recall (PR) curve and smoothed PR curve.
* Compute the mAP over the provided test data (a frame from the [Waymo Open Dataset](https://waymo.com/open)).

## 1. Introduction

In [ ]:
### Importing the required modules

In [1]:
from collections import Counter
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
from typing import List

In [ ]:
tf.__version__

In [ ]:
tf.test.gpu_device_name()

In [ ]:
### Setting the environment variables

In [ ]:
ENV_COLAB = False                # True if running in Google Colab instance

In [ ]:
# Root directory
DIR_BASE = '' if not ENV_COLAB else '/content/'

In [ ]:
# Subdirectory to save output files
DIR_OUT = os.path.join(DIR_BASE, 'out/')
# Subdirectory pointing to input data
DIR_SRC = os.path.join(DIR_BASE, 'data/')

In [ ]:
### Creating subdirectories (if not exists)
os.makedirs(DIR_OUT, exist_ok=True)

### 1.1. Mean Average Precision (mAP)

#### Background

[Mean Average Precision](https://en.wikipedia.org/wiki/Evaluation_measures_\(information_retrieval\)#Mean_average_precision) (mAP) is a widely-used accuracy metric for object detection models. As the name entails, mAP is simply the average of the [Average Precision](https://en.wikipedia.org/wiki/Evaluation_measures_\(information_retrieval\)#Average_precision) (AP) computed with respect to all classes.

#### Precision and Recall

[Precision](https://en.wikipedia.org/wiki/Precision_and_recall#Precision) and [recall](https://en.wikipedia.org/wiki/Precision_and_recall#Recall) (sometimes called [sensitivity](https://en.wikipedia.org/wiki/Sensitivity_and_specificity)) are two single-valued metrics often computed over the entire document set (in our case, the set of all predictions made). We can, however, also compute precision and recall at every step along that interval. 

In other words, we can compute these two metrics cumulatively, prediction-by-prediction.

##### Precision-Recall (PR) curve

We can use the cumulative precision and recall calculations to plot a Precision-Recall curve. In order to generate the points along this curve, we have to calculate the _true positive_, _false positive_, _true negative_ and _false negative_ rate over a set of of predictions and their ground truth labels:

$$
\begin{align*}
    \mathrm{Precision} &= \frac{\mathrm{TP}}{\mathrm{TP} + \mathrm{FP}} = \frac{\mathrm{TP}}{\mathrm{all \ detections}}, &
    \mathrm{Recall} &= \frac{\mathrm{TP}}{\mathrm{TP} + \mathrm{FN}} = \frac{\mathrm{TP}}{\mathrm{all \ ground \ truths}}.
\end{align*}
$$

Often times the resulting PR curve will have extremes (jagged peaks) and result in a less-accurate area under the curve (AUC) calculation. To fix this, two common [interpolation](https://en.wikipedia.org/wiki/Interpolation) techniques are used to smooth out the PR curve.

###### Interpolation

The [11-Point Interpolation](https://github.com/rafaelpadilla/Object-Detection-Metrics#11-point-interpolation) technique pioneered by Everingham et al., 2010 [1] attempts to summarise the shape of the Precision $x$ Recall curve by averaging the precision values at a set of eleven equally-spaced recall levels in the range $[0, 1]$,

$$
\begin{align*}
\mathrm{AP} &= \frac{1}{11} \sum_{r \ \in \ \{0, 0.1, .., 1\}} \rho_{\mathrm{interp}}(r), \quad
& \rho_{\mathrm{interp}}(r) &= \max_{\bar{r}:\bar{r} \geq r}(\tilde{r}).
\end{align*}
$$

where $\rho(\tilde{r})$ is the measured precision at recall $\tilde{r}$. The precision value at each observed point is not used to compute the PR curve, instead the $AP$ value is obtained by interpolation the precision only at these $r=11$ levels. The precision is determined to be the maximum precision value whose recall is greater than $r$.

The [All-Points Interpolation](https://github.com/rafaelpadilla/Object-Detection-Metrics#interpolating-all-points) [2] method extends the 11-Points method to a set of recall levels $n$ equal to the number of observations along the Precision $x$ Recall curve, such that

$$
\begin{align*}
\mathrm{AP} &= \frac{1}{n} \sum_{n=0} \left(r_{n+1} - r_{n}\right) \rho_{\mathrm{interp}}(r_{n+1}), \quad
& \rho_{\mathrm{interp}}(r_{n+1}) &= \max_{\bar{r}:\bar{r} \geq r_{n+1}}(\tilde{r}).
\end{align*}
$$

where $p(\tilde{r})$ is the measured precision at recall $\tilde{r}$. Instead of using precision observed at only $r = 11$ points, we can now obtain the average precision (AP) by interpolating at each level $r$ and taking the maximum precision value whose recall is greater than $r + 1$. 

## 2. Programming Task

### 2.1. Precision-Recall

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def calculate_iou(gt_bbox, pred_bbox):
    """
    calculate iou 
    args:
    - gt_bbox [array]: 1x4 single gt bbox
    - pred_bbox [array]: 1x4 single pred bbox
    returns:
    - iou [float]: iou between 2 bboxes
    """
    xmin = np.max([gt_bbox[0], pred_bbox[0]])
    ymin = np.max([gt_bbox[1], pred_bbox[1]])
    xmax = np.min([gt_bbox[2], pred_bbox[2]])
    ymax = np.min([gt_bbox[3], pred_bbox[3]])
    
    intersection = max(0, xmax - xmin + 1) * max(0, ymax - ymin + 1)
    gt_area = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])
    pred_area = (pred_bbox[2] - pred_bbox[0]) * (pred_bbox[3] - pred_bbox[1])
    
    union = gt_area + pred_area - intersection
    return intersection / union

In [ ]:
def calculate_recall():
    pass

In [ ]:
def calculate_precision():
    pass

In [ ]:
def create_precision_recall_curve():
    pass

To do so, you will first have to create the Precision-Recall (PR) curve. Once this curve is created, you need to create the smoothed version as discussed in the lesson. Finally you can use this smoothed version to calculate the mAP.

In [ ]:
# load data 
with open('data/predictions.json', 'r') as f:
    preds = json.load(f)
    
with open('data/ground_truths.json', 'r') as f:
        gts = json.load(f)

In [ ]:
# TODO IMPLEMENT THIS SCRIPT

You also have to create a visualization of the PR and smoothed PR curves.

Make sure to check the `Desktop` to see your visualization when running the code.

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def check_results(output):
    round_output = np.round(output * 1e4) / 1e4
    assert round_output == 0.7286, 'Something is wrong with the mAP calculation'
    print('mAP calculation is correct!')

In [ ]:
check_results(mAP)

## Tips

To create the PR curve, you need to sort the predictions based on their confidence score and calculate the precision and recall for each subset of the predictions, as explained in the course.

To make your life easier, you can hard code the smoothed PR curve based on the PR curve, but 
you should think of a scripted version of doing so.

References
* [1] Everingham, M., et al., The PASCAL Visual Object Classes (VOC) Challenge. International Journal of Computer Vision. 88:303–338. 2010. [doi:10.1007/s11263-009-0275-4](https://doi.org/10.1007/s11263-009-0275-4).

* [2] Padilla, R., et al., A Comparative Analysis of Object Detection Metrics with a Companion Open-Source Toolkit. Electronics. 10(3):279. [doi:10.3390/electronics10030279](https://www.mdpi.com/2079-9292/10/3/279).


Helpful resources:
* [A Coder's Guide to IoU, Non-Max Suppression and Mean Average Precision by Vijayabhaskar J. | Medium](https://vijayabhaskar96.medium.com/practitioners-guide-to-iou-non-max-suppression-and-mean-average-precision-e09de73a2bd8)